In [71]:
!pip install opendatasets pandas -q


In [73]:
import opendatasets as od
import pandas
#{"username":"khwrali","key":"a2b23dfbac2443ab4db34e48318ce4ff"}
od.download(
    "https://www.kaggle.com/datasets/sripaadsrinivasan/audio-mnist")

Skipping, found downloaded files in "./audio-mnist" (use force=True to force download)


In [74]:
import os
import librosa
import numpy as np
from tqdm import tqdm

In [75]:
data_folder = '/content/audio-mnist/data'

def extract_mfccs(file_path, num_mfcc=13):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=num_mfcc)
    return mfccs


In [76]:
num_mfcc = 13
num_instances = 50
num_speakers = 60
num_digits = 10

In [77]:
fixed_length = 100

mfccs_data = []
labels = []

for speaker_id in tqdm(range(1, num_speakers + 1), desc='Speakers'):
    speaker_folder = os.path.join(data_folder, f'{speaker_id:02d}')

    for digit in range(num_digits):
        for instance in tqdm(range(num_instances), desc=f'Digit {digit}'):
            file_name = f'{digit}_{speaker_id:02d}_{instance}.wav'
            file_path = os.path.join(speaker_folder, file_name)
            mfccs = extract_mfccs(file_path, num_mfcc)
            if mfccs.shape[1] < fixed_length:
                mfccs = np.pad(mfccs, ((0, 0), (0, fixed_length - mfccs.shape[1])))
            else:
                mfccs = mfccs[:, :fixed_length]

            mfccs_data.append(mfccs)
            labels.append(digit)

Streaming output truncated to the last 5000 lines.
Digit 8: 100%|██████████| 50/50 [00:00<00:00, 103.81it/s]

Digit 0: 100%|██████████| 50/50 [00:00<00:00, 106.60it/s]

Digit 1: 100%|██████████| 50/50 [00:00<00:00, 109.00it/s]

Digit 2: 100%|██████████| 50/50 [00:00<00:00, 106.10it/s]

Digit 3: 100%|██████████| 50/50 [00:00<00:00, 92.20it/s]

Digit 4: 100%|██████████| 50/50 [00:00<00:00, 102.61it/s]

Digit 5: 100%|██████████| 50/50 [00:00<00:00, 90.42it/s]

Digit 6: 100%|██████████| 50/50 [00:00<00:00, 91.09it/s]

Digit 7: 100%|██████████| 50/50 [00:00<00:00, 91.77it/s]

Digit 8: 100%|██████████| 50/50 [00:00<00:00, 106.06it/s]

Digit 0: 100%|██████████| 50/50 [00:00<00:00, 96.88it/s] 

Digit 1: 100%|██████████| 50/50 [00:00<00:00, 51.88it/s]

Digit 2: 100%|██████████| 50/50 [00:01<00:00, 49.45it/s]

Digit 3: 100%|██████████| 50/50 [00:00<00:00, 50.98it/s]

Digit 4: 100%|██████████| 50/50 [00:00<00:00, 60.22it/s]

Digit 5: 100%|██████████| 50/50 [00:00<00:00, 95.77it/s]

Digit 6: 100%|

In [78]:
mfccs_data = np.array(mfccs_data)
labels = np.array(labels)
print("MFCCs shape:", mfccs_data.shape)
print("Labels shape:", labels.shape)

MFCCs shape: (30000, 13, 100)
Labels shape: (30000,)


In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    mfccs_data, labels, test_size=0.1, random_state=42, stratify=labels
)

print("Train set shapes:", X_train.shape, y_train.shape)
print("Test set shapes:", X_test.shape, y_test.shape)


Train set shapes: (27000, 13, 100) (27000,)
Test set shapes: (3000, 13, 100) (3000,)


In [91]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(13, 100)))
model.add(Dense(10, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", test_accuracy)

predictions = model.predict(X_test)


Epoch 1/10
844/844 [==============================] - 23s 24ms/step - loss: 0.8038 - accuracy: 0.7413 - val_loss: 0.4305 - val_accuracy: 0.8667
Epoch 2/10
844/844 [==============================] - 21s 25ms/step - loss: 0.3610 - accuracy: 0.8867 - val_loss: 0.3733 - val_accuracy: 0.8773
Epoch 3/10
844/844 [==============================] - 25s 30ms/step - loss: 0.2856 - accuracy: 0.9110 - val_loss: 0.3039 - val_accuracy: 0.9040
Epoch 4/10
844/844 [==============================] - 22s 26ms/step - loss: 0.2354 - accuracy: 0.9265 - val_loss: 0.2762 - val_accuracy: 0.9113
Epoch 5/10
844/844 [==============================] - 20s 24ms/step - loss: 0.2039 - accuracy: 0.9367 - val_loss: 0.2496 - val_accuracy: 0.9223
Epoch 6/10
844/844 [==============================] - 22s 26ms/step - loss: 0.1914 - accuracy: 0.9377 - val_loss: 0.2531 - val_accuracy: 0.9227
Epoch 7/10
844/844 [==============================] - 21s 25ms/step - loss: 0.1839 - accuracy: 0.9393 - val_loss: 0.2453 - val_accuracy:

In [102]:
print(X_test[0].shape)
print(np.argmax(predictions[0]))

(13, 100)
1
